## Construct checkpoint
The script aims to recalculate the results from the existing csv data and construce it into the serialized data that could get access by ray.tune.

STEP1: calculate the result for a csv file

STEP2: find the file name of it and match it to the result of `searcher-state`

STEP3: rewrite the result

STEP4: move the folder of trial into the constructed folder

STEP5: go through the 3 data files for ray.tune and change some parameters like `bayesopt._total_random_search_trials`

STEP6: save the result for all csv seperately

After that: get the phase plane for c_ii = 2/9 to define search space
            then launch tuning algorithm

In [1]:
from ray.tune.suggest.bayesopt import BayesOptSearch
from ray import tune
import os
import json
import numpy as np
from utils import *
import re
import evaluation
from ray.tune.utils.serialization import TuneFunctionEncoder,TuneFunctionDecoder

In [2]:
old_folder = '/mnt/user/drive/My Libraries/tutorials&explorations/data/ray_results/20220614_trial1'
new_folder = '/mnt/user/drive/My Libraries/tutorials&explorations/data/ray_results/20220623_trial1'
csv_folder = '/mnt/user/drive/My Libraries/tutorials&explorations/data/'
result_names = get_result_names(csv_folder)
json_path = os.path.join(csv_folder, 'results_20220622.json')

In [3]:
bayesopt = BayesOptSearch(metric="score",
                          mode="min",
                          random_search_steps=150)
bayesopt.restore(os.path.join(old_folder, 'searcher-state-2022-06-20_16-57-26.pkl'))

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator GaussianProcessRegressor from version 1.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
csv_results = []

In [5]:
with open(json_path,'r') as jf:
    csv_results = json.load(jf)
    jf.close()

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/user/drive/My Libraries/tutorials&explorations/data/results_20220622.json'

In [ ]:
with open(os.path.join(old_folder,'experiment_state-2022-06-20_16-57-26.json')) as f:
    ckp=json.load(f)
checkpoint=ckp['checkpoints']
f.close()

In [6]:
len(bayesopt._buffered_trial_results)

150

In [72]:
def modify_string(string, target, first_location, last_location, piece):
    loc = string.find(target)
    old_piece = string[loc+first_location:loc+last_location]
    list_str = list(string)
    str_len = last_location-first_location
    try:
        assert (len(list(piece))==str_len), 'Length of both string should be the same!'
    except AssertionError:
        str_len = len(piece)
        last_location = first_location+str_len
    list_str[loc+first_location:loc+last_location] = piece
    new_string = ''.join(list_str)
    for match in re.finditer(old_piece,new_string):
        list_new_str = list(new_string)
        # import pdb; pdb.set_trace()
        list_new_str[match.span()[0]: match.span()[1]] =  piece
        new_string = ''.join(list_new_str)
    return new_string, old_piece

In [73]:
c_ii_all = []

In [74]:
# useless_exp = []
# useless_opt = []
useless_csv = []
preserve_exp = []
preserve_opt = []
for k in range(len(csv_results)):
    dfa_all = csv_results[k]['dfa']
    score = evaluation.get_score(dfa_all, csv_results[k]['peak'])
    csv_results[k]['score'] = score
    file = csv_results[k]['file']
    c_ee_csv = file[file.find('c_ee')+4:file.find('c_ee')+19]
    c_ei_csv = file[file.find('c_ei')+4:file.find('c_ei')+19]
    exp_findone = False
    opt_findone = False
    for i in range(len(checkpoint)):
        # import pdb;pdb.set_trace()
        c_ee_s = checkpoint[i].find('\n    "c_ee": ')
        c_ee_exp = checkpoint[i][c_ee_s+13:c_ee_s+13+15]
        c_ei_s = checkpoint[i].find('\n    "c_ei": ')
        c_ei_exp = checkpoint[i][c_ei_s+13:c_ei_s+13+15]
        if (c_ee_csv in c_ee_exp) & (c_ei_csv in c_ei_exp):
            exp_findone = True
            preserve_exp.append(i)
            checkpoint[i], _ = modify_string(checkpoint[i], '"_last_result\": {\n    \"score\":', 31, 31+18,str(score)[0:18])
            checkpoint[i], _ = modify_string(checkpoint[i], '\"delta\"', 9, 9+17,str(dfa_all['delta'])[0:17])
            checkpoint[i], _ = modify_string(checkpoint[i], '\"theta\"', 9, 9+17,str(dfa_all['theta'])[0:17])
            checkpoint[i], _ = modify_string(checkpoint[i], '\"alpha\"', 9, 9+17,str(dfa_all['alpha'])[0:17])
            checkpoint[i], _ = modify_string(checkpoint[i], '\"beta\"', 8, 8+17,str(dfa_all['beta'])[0:17])
            checkpoint[i], _ = modify_string(checkpoint[i], '\"gamma\"', 9, 9+17,str(dfa_all['gamma'])[0:17])
            checkpoint[i], _ = modify_string(checkpoint[i], '\"raw\"', 7, 7+17,str(dfa_all['raw'])[0:17])
            checkpoint[i], _ = modify_string(checkpoint[i], 'dfa_this_trial\":', 9, 9+17,str(dfa_all['dfa_this_trial'])[0:17])
            checkpoint[i], _ = modify_string(checkpoint[i], 'f_peak\": ', 9, 9+9,str(csv_results[k]['peak'])[0:9])
    for j in range(len(bayesopt._buffered_trial_results)):
        if (c_ee_csv in str(bayesopt._buffered_trial_results[j][0]['c_ee'])) \
        & (c_ei_csv in str(bayesopt._buffered_trial_results[j][0]['c_ei'])):
            opt_findone = True
            preserve_opt.append(j)
            bayesopt._buffered_trial_results[j][1]['score'] = score
            bayesopt._buffered_trial_results[j][1]['f_peak'] = csv_results[k]['peak']
            bayesopt._buffered_trial_results[j][1]['dfa_all/delta'] = dfa_all['delta']
            bayesopt._buffered_trial_results[j][1]['dfa_all/theta'] = dfa_all['theta']
            bayesopt._buffered_trial_results[j][1]['dfa_all/alpha'] = dfa_all['alpha']
            bayesopt._buffered_trial_results[j][1]['dfa_all/beta'] = dfa_all['beta']
            bayesopt._buffered_trial_results[j][1]['dfa_all/gamma'] = dfa_all['gamma']
            bayesopt._buffered_trial_results[j][1]['dfa_all/raw'] = dfa_all['raw']
            bayesopt._buffered_trial_results[j][1]['dfa_all/dfa_this_trial'] = dfa_all['dfa_this_trial']
            
            c_ii_all.append(bayesopt._buffered_trial_results[j][0]['c_ii'])
            
            # copy the folder of the result to the new folder
            model_name='run_model_'+str(bayesopt._buffered_trial_results[j][1]['trial_id'])
            old_list = os.listdir(path=old_folder)
            trans_name = [s for s in old_list if model_name in s]
            trans_path = os.path.join(old_folder, trans_name[0]).replace(" ", "\\ ").replace("&", "\\&")
            target_path = new_folder.replace(" ", "\\ ").replace("&", "\\&")
            os.system('cp -r '+trans_path+' '+target_path)
    if (exp_findone==False) & (opt_findone==False):
        useless_csv.append(k)

In [75]:
useless_csv

[17, 22, 35, 59, 71, 89]

In [88]:
useless_path = os.path.join(csv_folder, 'useless_traces').replace(" ", "\\ ").replace("&", "\\&")
# os.makedirs(useless_path)

for i in useless_csv:
    original_path = os.path.join(csv_folder,csv_results[i]['file']).replace(" ", "\\ ").replace("&", "\\&")
    os.system('mv '+original_path+' '+useless_path)

In [72]:
dir(bayesopt)

['CKPT_FILE_TMPL',
 'FINISHED',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_analysis',
 '_buffered_trial_results',
 '_config_counter',
 '_live_trial_mapping',
 '_metric',
 '_metric_op',
 '_mode',
 '_patience',
 '_points_to_evaluate',
 '_random_state',
 '_register_result',
 '_setup_optimizer',
 '_skip_duplicate',
 '_space',
 '_total_random_search_trials',
 '_verbose',
 'add_evaluated_point',
 'add_evaluated_trials',
 'convert_search_space',
 'get_state',
 'max_concurrent',
 'metric',
 'mode',
 'on_trial_complete',
 'on_trial_result',
 'optimizer',
 'random_search_trials',
 'register_analysis',
 'repeat_float_precision',
 'restore',
 'restore_from_dir',
 'save',
 '

In [92]:
preserve_opt

[105,
 134,
 117,
 139,
 123,
 145,
 133,
 140,
 109,
 27,
 131,
 111,
 114,
 107,
 112,
 113,
 127,
 106,
 125,
 28,
 122,
 121,
 143,
 128,
 108,
 1,
 110,
 109,
 114,
 141,
 111]

In [100]:
buffered_len = len(bayesopt._buffered_trial_results)
useless_opt = list(set(list(range(buffered_len))).difference(set(preserve_opt)))

In [101]:
expe_len = len(checkpoint)
useless_exp = list(set(list(range(expe_len))).difference(set(preserve_exp)))

In [102]:
results = np.array(bayesopt._buffered_trial_results)
new_results = np.delete(results, useless_opt,axis=0)
bopt = bayesopt
bopt._buffered_trial_results = new_results #.tolist()?

In [105]:
bopt.random_search_trials = 200

In [106]:
bopt._total_random_search_trials=len(new_results)

In [107]:
bopt.save(os.path.join(new_folder, 'searcher-state-2022-06-23_14-00-00.pkl'))

In [109]:
def array2tuple(x):
    return (x[0],x[1])

new_buffer = list(map(array2tuple,new_results))
len(new_buffer)

In [108]:
ckpt_array = np.array(checkpoint)
new_ckpt_array = np.delete(ckpt_array,useless_exp)

In [110]:
new_ckpt=list(map(str,new_ckpt_array))

In [111]:
len(new_ckpt)

91

In [14]:
fp = '/mnt/user/drive/My Libraries/tutorials&explorations/data/ray_results/new_folder/experiment_state-2022-06-23_14-00-00.json'

In [34]:
ckp['checkpoints'] = new_ckpt

In [35]:
ckp['runner_data']['checkpoint_file'] = fp

In [36]:
ckp['stats']

{'start_time': 1655469387.466305, 'timestamp': 1655547700.0619483}

In [37]:
with open(fp, "w") as ckpf:
    json.dump(ckp, ckpf, indent=2, cls=TuneFunctionEncoder)
ckpf.close()

In [2]:
from ray.tune.suggest.suggestion import ConcurrencyLimiter, Searcher
from ray.tune.suggest import SearchGenerator
search_alg = BayesOptSearch(metric="score",
                          mode="min",
                          random_search_steps=150)
max_concurrent_trials = 3
search_alg = ConcurrencyLimiter(search_alg, max_concurrent=max_concurrent_trials)

In [3]:
if isinstance(search_alg, Searcher):
        search_alg = SearchGenerator(search_alg)

In [6]:
search_alg.restore_from_dir(new_folder)

In [10]:
CKPT_FILE_TMPL = "search_gen_state-{}.json"
search_alg_state = tune.utils.util.load_newest_checkpoint(
            dirpath, CKPT_FILE_TMPL.format("*")
        )

In [ ]:
dir(search_alg_state)

In [23]:
search_alg_state['counter']=143

In [25]:
tune.utils.util.atomic_save(state=search_alg_state,checkpoint_dir=dirpath,file_name='search_gen_state-2022-06-23_14-00-00.json',tmp_file_name='search_gen_state-2022-06-23_14-00-00.json')

In [38]:
with open(fp) as f:
    runner_state = json.load(f, cls=TuneFunctionDecoder)
    f.close()

In [39]:
len(runner_state['checkpoints'])

143